In [33]:
import pandas as pd

In [34]:
all_data = pd.read_csv("all.leaf.thermistor.data.csv")
all_data.head()

,timestamp,date,time,hour,minute,logger.id,tree.id,leaf,population,temp
0,7/24/23 17:00,7/24/23,17:00:00,17,0,1,67,1,JLA,49.92
1,7/24/23 17:30,7/24/23,17:30:00,17,30,1,67,1,JLA,48.14
2,7/24/23 18:00,7/24/23,18:00:00,18,0,1,67,1,JLA,47.26
3,7/24/23 18:30,7/24/23,18:30:00,18,30,1,67,1,JLA,44.34
4,7/24/23 19:00,7/24/23,19:00:00,19,0,1,67,1,JLA,42.06


In [35]:
all_data.columns

Index(['timestamp', 'date', 'time', 'hour', 'minute', 'logger.id', 'tree.id',
       'leaf', 'population', 'temp'],
      dtype='object')

In [36]:
all_data = all_data.drop(['logger.id', 'tree.id', 'leaf', 'minute', 'time', 'hour', 'date'], axis=1)

In [37]:
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'], format="%m/%d/%y %H:%M")

In [38]:
all_data['timestamp'] = all_data['timestamp'].dt.floor('h')

In [39]:
all_data = all_data.rename(columns={'temp' : 'leaftemp'})

In [40]:
populations = ['CCR', 'JLA', 'NRV', 'TSZ']

In [41]:
def timestamp_to_cols(df):
    # Add Year column
    df["year"] = df['timestamp'].apply(lambda x: x.strftime("%Y")).astype(int)
    df["julian-day"] = df['timestamp'].apply(lambda x: x.strftime("%j")).astype(int)
    df['standard-time'] = df['timestamp'].apply(lambda x: x.strftime("%H")).astype(int)

    return df

In [42]:
for pop in populations:
    leaft = all_data[all_data['population'] == pop].groupby(['timestamp', 'population']).mean(numeric_only=True)
    leaft = leaft.reset_index().drop(['population'], axis=1)
    leaft = timestamp_to_cols(leaft).drop(columns=['timestamp']).reindex(columns=['year', 'julian-day', 'standard-time', 'leaftemp'])
    leaft.to_csv(f"{pop}_leaftemp.csv", index=False)
    hourly = pd.read_csv(f"{pop}_hourly_data.csv")
    # Align leaftemp to the hourly index using a merge on year, julian-day, and standard-time
    hourly = hourly.merge(
        leaft[['year', 'julian-day', 'standard-time', 'leaftemp']],
        on=['year', 'julian-day', 'standard-time'],
        how='left'
    )
    hourly.to_csv(f"{pop}_hourly_data.csv", index=False)